In [1]:
import pandas as pd
import numpy as np
import copy
import os
import sys

In [2]:
df = pd.read_csv("/mnt/cfs/zhanbojun/pykt-toolkit/data/assist2009/train_valid.csv")

In [3]:
df['uid'].nunique()

3082

In [4]:
ALL_KEYS = ["fold", "uid", "questions", "concepts", "responses", "timestamps",
            "usetimes", "selectmasks", "is_repeat", "qidxs", "rest", "orirow", "cidxs"]
ONE_KEYS = ["fold", "uid","uni_concepts_num"]


def calStatistics(df, stares, key):
    allin, allselect = 0, 0
    allqs, allcs = set(), set()
    for i, row in df.iterrows():
        rs = row["responses"].split(",")
        curlen = len(rs) - rs.count("-1")
        allin += curlen
        if "selectmasks" in row:
            ss = row["selectmasks"].split(",")
            slen = ss.count("1")
            allselect += slen
        if "concepts" in row:
            cs = row["concepts"].split(",")
            fc = list()
            for c in cs:
                cc = c.split("_")
                fc.extend(cc)
            curcs = set(fc) - {"-1"}
            allcs |= curcs
        if "questions" in row:
            qs = row["questions"].split(",")
            curqs = set(qs) - {"-1"}
            allqs |= curqs
    stares.append(",".join([str(s)
                  for s in [key, allin, df.shape[0], allselect]]))
    return allin, allselect, len(allqs), len(allcs), df.shape[0]


def save_dcur(row, effective_keys):
    dcur = dict()
    for key in effective_keys:
        if key not in ONE_KEYS:
            dcur[key] = row[key].split(",")#[int(i) for i in row[key].split(",")]
        else:
            dcur[key] = row[key]
    return dcur

In [5]:
stares = []
ins, ss, qs, cs, seqnum = calStatistics(df, stares, 'origin train valid')
print(
        f"train+valid original interactions num: {ins}, select num: {ss}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")

train+valid original interactions num: 274614, select num: 0, qs: 17405, cs: 120, seqnum: 3082


# 固定KC序列

In [9]:
def process_row(row):
    cs = row['concepts'].split(",")
    fc = set()
    for c in cs:
        cc = c.split("_")
        fc.update(cc)
    row['uni_concepts_num'] = len(fc)
    row['uni_concepts'] = ",".join(fc)
    return row
    

In [10]:
df2 = df.apply(process_row, axis=1)

In [11]:
# 按照uni_concepts_num排序,并更新索引,降序
df2 = df2.sort_values(by='uni_concepts_num', ascending=False).reset_index(drop=True)

In [12]:
df3 = pd.DataFrame(columns=df.columns)
cur_max = 0

In [13]:

def gredy_select(df, seen_concepts, drop_index,cur_max):
    global df3
    cur_max_index = None
    drop_index_len = len(drop_index)
    for index, row in df.iterrows():
        if index in drop_index:
            continue
        uc = row['uni_concepts'].split(",")
        uc = set(uc)
        if len(uc - seen_concepts) > cur_max:
            cur_max = len(uc - seen_concepts)
            cur_max_index = index
        # print(cur_max_index)
    if cur_max_index is not None:
        seen_concepts.update(df.loc[cur_max_index]['uni_concepts'].split(","))
        df3 = pd.concat([df3, pd.DataFrame(df.loc[cur_max_index]).T])
        drop_index.append(cur_max_index)
        #print(f"drop_index:{drop_index}")
    cur_drop_index_len = len(drop_index)
    if len(seen_concepts) != 120 and cur_drop_index_len != drop_index_len:
        return gredy_select(df, seen_concepts, drop_index, 0)
    elif cur_drop_index_len == drop_index_len:
        print("已遍历所有数据，不满足全部知识点覆盖条件，当前知识点覆盖数为：", len(seen_concepts))
        return df3, drop_index
    else:
        print("已满足全部知识点覆盖条件，当前知识点覆盖数为：", len(seen_concepts))
        return df3, drop_index

In [ ]:
for i in range (16):
    df3, drop_index = gredy_select(df2, set(), [],0)
    df2 = df2.drop(drop_index)

# 固定交互序列个数

In [6]:
df.head()

,fold,uid,questions,concepts,responses,is_repeat
0,0,3675,"11666,8406,11471,10543,7571,15890,8117,11023,1...","62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,1...","0,0,0,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
1,0,3651,"2941,11778,1141,1282,3287,3411,840,1952,11540","1,1,1,1,1,1,1,1,1","0,0,1,1,1,1,0,0,1","0,0,0,0,0,0,0,0,0"
2,0,1692,"10349,11182,7926,7900,7906,7137,10923","57,57,57,57,57,57,57","1,0,1,1,1,1,1","0,0,0,0,0,0,0"
3,0,3076,"2549,830,4313,1826,15138,4687,4632,13791,15137...","15,15,15,15,14,14,14,14,14,14,14","0,1,1,0,0,1,1,0,1,0,1","0,0,0,0,0,0,0,0,0,0,0"
4,0,1558,"7927,7928,10084,10923,15540,15541,15542,15543,...","57,57,57,57,57,57,57,57,57,57,57,57,57,48","0,0,1,0,0,0,0,1,1,1,1,1,1,0","0,0,0,0,0,0,0,0,0,0,0,0,0,0"


In [17]:
def process_row(row):
    rs = row["responses"].split(",")
    row['interaction_num'] = len(rs)
    return row

In [26]:
df2 = df.apply(process_row, axis=1)
df2 = df2.sample(frac=1, random_state=42).reset_index(drop=True)

In [39]:
# 遍历df2，每一行的interaction_num累加，如果总数不大于1000，则加入df3,
df3 = pd.DataFrame(columns=df.columns)

def get_df3(df2, int_num):
    cur_sum = 0
    global df3
    for index, row in df2.iterrows():
        cur_sum += row['interaction_num']
        if cur_sum <= int_num:
            df3 = pd.concat([df3, pd.DataFrame(row).T])
        else:
            break
    return df3

In [40]:
df3 = get_df3(df2, 5364)

In [41]:
stares = []
ins, ss, qs, cs, seqnum = calStatistics(df3, stares, 'origin train valid')
print(
        f"train+valid original interactions num: {ins}, select num: {ss}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")

train+valid original interactions num: 5326, select num: 0, qs: 3440, cs: 99, seqnum: 72


# 切分sequences

In [15]:
def KFold_split(df, k=5):
    df = df.sample(frac=1.0, random_state=1024)
    datanum = df.shape[0]
    test_ratio = 1 / k
    test_num = int(datanum * test_ratio)
    rest = datanum % k

    start = 0
    folds = []
    for i in range(0, k):
        if rest > 0:
            end = start + test_num + 1
            rest -= 1
        else:
            end = start + test_num
        folds.extend([i] * (end - start))
        print(f"fold: {i+1}, start: {start}, end: {end}, total num: {datanum}")
        start = end
    # report
    finaldf = copy.deepcopy(df)
    finaldf["fold"] = folds
    return finaldf

In [16]:
df3 = KFold_split(df3, 5)
stares = []

fold: 1, start: 0, end: 12, total num: 58
fold: 2, start: 12, end: 24, total num: 58
fold: 3, start: 24, end: 36, total num: 58
fold: 4, start: 36, end: 47, total num: 58
fold: 5, start: 47, end: 58, total num: 58


In [130]:
stares = []
ins, ss, qs, cs, seqnum = calStatistics(df3, stares, 'origin train valid')
print(
        f"train+valid original interactions num: {ins}, select num: {ss}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")

train+valid original interactions num: 457328, select num: 0, qs: 11670, cs: 188, seqnum: 107


In [145]:
def generate_sequences(df, effective_keys, min_seq_len=3, maxlen=200, pad_val=-1):
    save_keys = list(effective_keys) + ["selectmasks"]
    dres = {"selectmasks": []}
    dropnum = 0
    for i, row in df.iterrows():
        dcur = save_dcur(row, effective_keys)

        rest, lenrs = len(dcur["responses"]), len(dcur["responses"])
        j = 0
        while lenrs >= j + maxlen:
            rest = rest - (maxlen)
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    # [str(k) for k in dcur[key][j: j + maxlen]]))
                    dres[key].append(",".join(dcur[key][j: j + maxlen]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * maxlen))

            j += maxlen
        if rest < min_seq_len:  # delete sequence len less than min_seq_len
            dropnum += rest
            continue

        pad_dim = maxlen - rest
        for key in effective_keys:
            dres.setdefault(key, [])
            if key not in ONE_KEYS:
                paded_info = np.concatenate(
                    [dcur[key][j:], np.array([pad_val] * pad_dim)])
                dres[key].append(",".join([str(k) for k in paded_info]))
            else:
                dres[key].append(dcur[key])
        dres["selectmasks"].append(
            ",".join(["1"] * rest + [str(pad_val)] * pad_dim))

    # after preprocess data, report
    dfinal = dict()
    for key in ALL_KEYS:
        if key in save_keys:
            dfinal[key] = dres[key]
    finaldf = pd.DataFrame(dfinal)
    print(f"dropnum: {dropnum}")
    return finaldf

In [146]:
effective_keys = df3.keys()
split_seqs = generate_sequences(
    df3, effective_keys=effective_keys)
ins, ss, qs, cs, seqnum = calStatistics(
    split_seqs, stares, "train+valid sequences")
print(
    f"train+valid sequences interactions num: {ins}, select num: {ss}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")

dropnum: 5
train+valid sequences interactions num: 457323, select num: 457323, qs: 11670, cs: 188, seqnum: 2336


In [147]:
split_seqs.head(-10)

,fold,uid,questions,concepts,responses,timestamps,usetimes,selectmasks,is_repeat
0,0,3246,"1274,175,1633,1158,514,3223,7721,455,455,455,4...","174,44,26,37,102,10,9,134,50,13,3,135,50,21,49...","0,0,1,1,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,...","1566997224500,1566997241416,1566997260612,1566...","24000,12000,13000,14000,34000,18000,32000,2600...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,0,1,..."
1,0,3246,"2042,2042,2042,2043,2043,2043,718,718,718,718,...","79,19,4,58,19,4,13,83,3,16,21,57,3,16,21,59,3,...","1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,...","1567333058285,1567333058285,1567333058285,1567...","33666,33666,33666,33666,33666,33666,28333,2833...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,0,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,..."
2,0,3246,"1749,1749,6472,6472,6472,6472,6472,4029,4027,4...","16,3,12,122,123,21,3,39,27,42,11,39,42,44,8,11...","1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,...","1567515325512,1567515325512,1567515352472,1567...","19000,19000,17000,17000,17000,17000,17000,2300...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,..."
3,0,3246,"675,675,675,675,675,675,676,676,676,676,676,67...","65,66,13,67,48,67,45,65,66,13,67,46,67,45,65,6...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1567687154478,1567687154478,1567687154478,1567...","47000,47000,47000,47000,47000,47000,47000,4700...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,..."
4,0,3246,"1291,1291,1291,1291,2926,2926,2926,5984,5984,5...","141,54,13,4,53,54,3,134,50,13,4,132,50,19,53,5...","1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,...","1568550924797,1568550924797,1568550924797,1568...","21000,21000,21000,21000,21000,21000,21000,2800...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,0,1,1,..."
...,...,...,...,...,...,...,...,...,...
2321,4,1709,"2542,1572,1572,1572,1572,5630,5630,5630,5630,4...","3,1,128,16,3,1,92,16,3,158,10,39,38,13,44,13,3...","1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,0,0,1,1,...","1564580034551,1564580080729,1564580080729,1564...","12000,19000,19000,19000,19000,17000,17000,1700...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,0,1,1,1,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,..."
2322,4,1709,"4244,4244,4242,4242,4245,4245,4225,4224,4226,4...","13,158,13,7,13,39,7,42,158,39,39,11,26,7,13,26...","0,0,1,1,1,1,0,1,1,0,1,0,1,1,1,1,1,1,0,0,0,0,1,...","1564729464742,1564729464742,1564729464781,1564...","22500,22500,22500,22500,22500,22500,30000,3000...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,..."
2323,4,1709,"1333,1336,1336,1334,1334,4263,4263,4262,4262,4...","26,13,39,13,11,13,39,13,8,13,34,13,42,13,26,13...","1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,...","1564843112436,1564843112454,1564843112454,1564...","40750,40750,40750,40750,40750,18500,18500,1850...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,..."
2324,4,1709,"943,943,943,943,945,945,945,945,946,946,946,94...","161,170,48,163,161,170,168,163,161,170,46,163,...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,...","1564917658470,1564917658470,1564917658470,1564...","62600,62600,62600,62600,62600,62600,62600,6260...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,0,1,..."


In [148]:
split_seqs.to_csv("/mnt/cfs/zhanbojun/pykt-toolkit/data/ednet/train_valid_sequences_greed_32.csv", index=False)

In [149]:
df = pd.read_csv("/mnt/cfs/zhanbojun/pykt-toolkit/data/nips_task/test_sequences.csv")

In [150]:
# 从df中选20%的数据保存为df2
df2 = df.sample(frac=0.2, random_state=1024)

In [151]:
df2.head()

,fold,uid,questions,concepts,responses,timestamps,usetimes,selectmasks,is_repeat,cidxs
516,-1,3016,"530,530,532,532,1083,1083,1084,1084,1082,1082,...","13,11,13,152,13,10,13,158,13,39,13,34,39,107,1...","1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,1,1,1,...","1516027372287,1516027372287,1516027374975,1516...","23500,23500,23500,23500,52250,52250,52250,5225...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,...","67862,67863,67864,67865,67866,67867,67868,6786..."
1833,-1,2905,"4571,2470,426,5390,517,4955,4957,891,8183,1184...","40,28,38,38,40,26,7,87,10,156,7,114,129,94,7,3...","1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,0,1,-1,-1,-1,-1,...","1541467947028,1541467963983,1541467982149,1541...","24000,13000,15000,23000,15000,25000,28000,2600...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,...","231408,231409,231410,231411,231412,231413,2314..."
1243,-1,4400,"365,365,366,366,366,366,366,367,367,367,367,36...","19,3,12,14,15,21,16,1,112,3,16,124,125,21,19,1...","1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,...","1501153644120,1501153644120,1501153662009,1501...","17000,17000,16000,16000,16000,16000,16000,1600...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,...","157320,157321,157322,157323,157324,157325,1573..."
1412,-1,2282,"815,813,813,813,813,813,814,814,814,814,814,81...","67,161,71,66,48,67,161,71,66,48,67,161,71,66,1...","1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...","1565088953941,1565088953957,1565088953957,1565...","76200,76200,76200,76200,76200,76200,76200,7620...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,...","177454,177455,177456,177457,177458,177459,1774..."
1351,-1,1026,"3531,3627,3627,3627,3626,3626,3626,3626,3625,3...","19,83,19,21,13,81,19,21,57,19,21,59,21,3,13,79...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,...","1534166272807,1534166345857,1534166345857,1534...","26666,21333,21333,21333,21333,21333,21333,2133...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,...","170537,170538,170539,170540,170541,170542,1705..."


In [152]:
df2.to_csv("/mnt/cfs/zhanbojun/pykt-toolkit/data/ednet/test_sequences_subset.csv", index=False)